In [0]:
# Ingestão de Sets - Magic: The Gathering
# Objetivo: Ingerir dados de sets da API do Magic: The Gathering para staging em Parquet no S3
# Características: Dados brutos, formato Parquet, filtro temporal, particionamento, incremental, tratamento de campos complexos

# =============================================================================
# BIBLIOTECAS UTILIZADAS
# =============================================================================
import requests
import json
import logging
from datetime import datetime, timedelta
from pyspark.sql.functions import current_timestamp, lit, col, year, month, when
from pyspark.sql.types import *
import time

# =============================================================================
# CONFIGURAÇÃO DE SEGREDOS
# =============================================================================

def get_secret(secret_name, default_value=None):
    try:
        return dbutils.secrets.get(scope="mtg-pipeline", key=secret_name)
    except:
        if default_value is not None:
            print(f"Segredo '{secret_name}' não encontrado, usando valor padrão")
            return default_value
        else:
            print(f"Segredo obrigatório '{secret_name}' não encontrado")
            raise Exception(f"Segredo '{secret_name}' não configurado")

# =============================================================================
# CONFIGURAÇÕES GLOBAIS
# =============================================================================

# Configuração de logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Configurações da API
API_BASE_URL = get_secret("api_base_url")
BATCH_SIZE = int(get_secret("batch_size", "100"))
MAX_RETRIES = int(get_secret("max_retries", "3"))

# Configurações do S3
S3_BUCKET = get_secret("s3_bucket")
S3_PREFIX = get_secret("s3_prefix")
S3_BASE_PATH = f"s3://{S3_BUCKET}/{S3_PREFIX}"

# Configurações de período
YEARS_BACK = int(get_secret("years_back", "5"))
current_year = datetime.now().year
cutoff_year = current_year - YEARS_BACK
CUTOFF_DATE = datetime(cutoff_year, 1, 1)
CUTOFF_DATE_STR = CUTOFF_DATE.strftime("%Y-%m-%d")

# Log das configurações
print("=" * 60)
print("CONFIGURAÇÕES PARA INGESTÃO DE SETS")
print("=" * 60)
print("API_BASE_URL: [CONFIGURADO]")
print("S3_BASE_PATH: [CONFIGURADO]")
print(f"YEARS_BACK: {YEARS_BACK}")
print(f"CUTOFF_DATE_STR: {CUTOFF_DATE_STR}")
print("=" * 60)


In [0]:
# =============================================================================
# FUNÇÕES UTILITÁRIAS
# =============================================================================

def setup_s3_storage():
    try:
        # Verificar se o diretório existe (criar se necessário)
        try:
            dbutils.fs.ls(S3_BASE_PATH)
            print("Diretório do S3 já existe")
        except:
            dbutils.fs.mkdirs(S3_BASE_PATH)
            print("Diretório do S3 criado com sucesso")
        
        return True
        
    except Exception as e:
        print(f"Erro ao configurar S3 storage: {e}")
        return False

def make_api_request(endpoint, params=None, retries=MAX_RETRIES):
    url = f"{API_BASE_URL}/{endpoint}"
    
    for attempt in range(retries):
        try:
            response = requests.get(url, params=params, timeout=30)
            
            if response.status_code == 200:
                return response.json()
            elif response.status_code == 429:  # Rate limit
                wait_time = min((attempt + 1) * 5, 60)
                print(f"Rate limit atingido. Aguardando {wait_time}s...")
                time.sleep(wait_time)
            elif response.status_code == 503:  # Service unavailable
                wait_time = min((attempt + 1) * 10, 120)
                print(f"Serviço indisponível. Aguardando {wait_time}s...")
                time.sleep(wait_time)
            else:
                print(f"Erro {response.status_code} na API")
                if attempt < retries - 1:
                    time.sleep(5)
                
        except requests.exceptions.Timeout:
            print(f"Timeout na tentativa {attempt + 1}")
            if attempt < retries - 1:
                time.sleep(10)
        except requests.exceptions.RequestException as e:
            if attempt == retries - 1:
                print(f"Erro na requisição para endpoint após {retries} tentativas: {e}")
                return None
            print(f"Tentativa {attempt + 1} falhou, tentando novamente...")
            time.sleep(1)
    
    return None

def clean_sets_data(data):
    cleaned_data = []
    for item in data:
        if isinstance(item, dict):
            cleaned_item = {}
            
            # Mapear campos conhecidos com tipos seguros
            field_mappings = {
                'code': str,
                'name': str,
                'type': str,
                'border': str,
                'mkm_id': int,
                'mkm_name': str,
                'releaseDate': str,
                'gathererCode': str,
                'magicCardsInfoCode': str,
                'oldCode': str,
                'onlineOnly': bool,
                'source': str
            }
            
            # Processar campos conhecidos
            for field, field_type in field_mappings.items():
                if field in item:
                    try:
                        if item[field] is not None:
                            cleaned_item[field] = field_type(item[field])
                        else:
                            cleaned_item[field] = None
                    except (ValueError, TypeError):
                        # Se não conseguir converter, usar string
                        cleaned_item[field] = str(item[field]) if item[field] is not None else None
                else:
                    cleaned_item[field] = None
            
            # Tratar campo booster complexo - EXPLODIR EM MÚLTIPLAS COLUNAS
            if 'booster' in item and item['booster'] is not None:
                booster_data = item['booster']
                
                if isinstance(booster_data, list):
                    # Para cada elemento da lista booster, criar uma coluna separada
                    for i, booster_item in enumerate(booster_data):
                        if isinstance(booster_item, list):
                            # Se o item é uma lista, converter para string JSON
                            cleaned_item[f'booster_{i}'] = json.dumps(booster_item)
                        else:
                            # Se é um valor simples, converter para string
                            cleaned_item[f'booster_{i}'] = str(booster_item)
                    
                    # Adicionar campo booster original como string JSON para referência
                    cleaned_item['booster'] = json.dumps(booster_data)
                else:
                    # Se não é lista, manter como string
                    cleaned_item['booster'] = str(booster_data)
            else:
                cleaned_item['booster'] = None
            
            cleaned_data.append(cleaned_item)
    
    return cleaned_data

def save_to_parquet(data, table_name):
    if not data:
        print(f"Nenhum dado para salvar na tabela {table_name}")
        return None
    
    try:
        # Criar DataFrame com schema explícito para sets
        if table_name == 'sets':
            # Schema explícito para sets com colunas booster explodidas
            schema_fields = [
                StructField("code", StringType(), True),
                StructField("name", StringType(), True),
                StructField("type", StringType(), True),
                StructField("border", StringType(), True),
                StructField("mkm_id", IntegerType(), True),
                StructField("mkm_name", StringType(), True),
                StructField("releaseDate", StringType(), True),
                StructField("gathererCode", StringType(), True),
                StructField("magicCardsInfoCode", StringType(), True),
                StructField("booster", StringType(), True),  # Campo original como JSON
                StructField("oldCode", StringType(), True),
                StructField("onlineOnly", BooleanType(), True),
                StructField("source", StringType(), True)
            ]
            
            # Adicionar colunas booster explodidas (até 20 posições para cobrir a maioria dos casos)
            for i in range(20):
                schema_fields.append(StructField(f"booster_{i}", StringType(), True))
            
            schema = StructType(schema_fields)
            df = spark.createDataFrame(data, schema)
        else:
            # Para outras tabelas, usar inferência automática
            df = spark.createDataFrame(data)
        
        # Adicionar metadados de ingestão
        df = df.withColumn("ingestion_timestamp", current_timestamp()) \
               .withColumn("source", lit("mtg_api")) \
               .withColumn("endpoint", lit(table_name))
        
        # Particionamento por Ano/Mês para sets
        if table_name == 'sets' and 'releaseDate' in df.columns:
            df = df.withColumn("partition_year", 
                              when(col("releaseDate").isNotNull(), 
                                   year(col("releaseDate")))
                              .otherwise(lit(datetime.now().year))) \
                   .withColumn("partition_month", 
                              when(col("releaseDate").isNotNull(), 
                                   month(col("releaseDate")))
                              .otherwise(lit(datetime.now().month)))
            
            # Filtrar apenas sets dos últimos 5 anos
            total_sets = df.count()
            df = df.filter(col("releaseDate") >= lit(CUTOFF_DATE_STR))
            filtered_sets = df.count()
            print(f"Sets filtrados: {filtered_sets}/{total_sets}")
        else:
            # Para outras tabelas, usar data atual
            df = df.withColumn("partition_year", lit(datetime.now().year)) \
                   .withColumn("partition_month", lit(datetime.now().month))
        
        # Salvar como Parquet no S3 com particionamento
        partition_combinations = df.select("partition_year", "partition_month").distinct().collect()
        
        for partition_row in partition_combinations:
            partition_year = partition_row["partition_year"]
            partition_month = partition_row["partition_month"]
            
            # Filtrar dados da partição
            partition_df = df.filter((col("partition_year") == partition_year) & 
                                   (col("partition_month") == partition_month))
            
            # Nome do arquivo
            file_name = f"{partition_year}_{partition_month:02d}_{table_name}.parquet"
            file_path = f"{S3_BASE_PATH}/{file_name}"
                
            # Verificar se arquivo já existe
            try:
                existing_files = dbutils.fs.ls(file_path)
                if len(existing_files) > 0:
                    print(f"Arquivo {file_name} já existe - pulando")
                    continue
            except:
                pass
            
            # Salvar arquivo
            partition_df.drop("partition_year", "partition_month").write.mode("overwrite").format("parquet").save(file_path)
            print(f"Arquivo {file_name} criado com sucesso")
        
        print(f"Registros salvos como Parquet para {table_name}")
        return df
        
    except Exception as e:
        print(f"Erro ao salvar dados em {table_name}: {e}")
        return None

def ingest_simple_data(endpoint, table_name, data_key=None):
    print(f"Iniciando ingestão simples: {table_name}")
    
    if data_key is None:
        data_key = table_name
    
    data = make_api_request(endpoint)
    
    if data and data_key in data:
        table_data = data[data_key]
        
        # Limpar dados se for sets
        if table_name == 'sets':
            print("Limpando dados de sets...")
            table_data = clean_sets_data(table_data)
            
            # Mostrar exemplo de explosão do booster
            if table_data and len(table_data) > 0:
                example_item = table_data[0]
                booster_fields = [k for k in example_item.keys() if k.startswith('booster_')]
                print(f"Campo booster explodido em {len(booster_fields)} colunas: {booster_fields[:5]}...")
        
        df = save_to_parquet(table_data, table_name)
        
        if df:
            count = df.count()
            print(f"{table_name}: {count} registros processados")
            display(df.limit(5))
        return df
    else:
        print(f"Falha ao obter dados de {table_name}")
        return None



# Configurar S3 Storage
setup_success = setup_s3_storage()
if not setup_success:
    raise Exception("Falha ao configurar S3 storage")

print("Setup concluído com sucesso")


In [0]:
# Iniciar ingestão de sets
print("Iniciando ingestão de sets...")

sets_df = ingest_simple_data(
    endpoint="sets",
    table_name="sets"
)



# Gerar relatório
print("=" * 50)
print("RELATÓRIO DE INGESTÃO DE SETS")
print("=" * 50)

if sets_df:
    print("Arquivos salvos")

else:
    print("Falha na ingestão de sets")

print("=" * 50)